In [ ]:
import numpy as np
import uproot as ur
import pandas as pd
import awkward as ak

import matplotlib.pyplot as plt

from myenv.icarusplot.icarusplot import *

import matplotlib
matplotlib.style.use('~/ICARUS_GUNDAM_Workshop/myenv/icarusplot/icarus_style.mplstyle')

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

# Let's look at our file

In [ ]:
#with ur.open('demo.root') as f:
#    print(f.keys())

# While GUNDAM wants the TClonesArrays, we will use the nomal one so we can draw the systematics. Let's get all the trees

In [ ]:
with ur.open('demo.root') as f:
    # Selected tree
    selected=f['selectedTree/Systematics']
    branchSel=[i.name for i in selected.branches]
    selectionTree=selected.arrays(branchSel,library='ak')
    # Truth tree
    truth=f['truthTree/Truth']
    branchTrue=[i.name for i in truth.branches]
    truthTree=truth.arrays(branchTrue,library='ak')
    # CCQE covariance info
    # -- tree
    truthCCQE=f['truthTreeCCQE/TruthCCQE']
    branchTrueCCQE=[i.name for i in truthCCQE.branches]
    truthTreeCCQE=truthCCQE.arrays(branchTrueCCQE,library='ak')

### Plot the selected signal (category=1) and backgrounds (OOPS=2, OtherCC=3, NuNC=4, Other=5)

In [ ]:
sel_signal = selectionTree[ ak.where( selectionTree['kNuMISliceSignalType']==1 ) ]
sel_oops = selectionTree[ ak.where( selectionTree['kNuMISliceSignalType']==2 ) ]
sel_othercc = selectionTree[ ak.where( selectionTree['kNuMISliceSignalType']==3 ) ]
sel_nunc = selectionTree[ ak.where( selectionTree['kNuMISliceSignalType']==4 ) ]
sel_other = selectionTree[ ak.where( selectionTree['kNuMISliceSignalType']==5 ) ]

In [ ]:
fig, ax = plt.subplots()
binsCosThMuP = np.linspace(-1.,1.,21)

ax.hist( selectionTree['kRecoCosThMuP'], bins=binsCosThMuP, histtype='step' )
ax.hist( [sel_signal['kRecoCosThMuP'], sel_oops['kRecoCosThMuP'], sel_othercc['kRecoCosThMuP'],\
          sel_nunc['kRecoCosThMuP'], sel_other['kRecoCosThMuP']], bins=binsCosThMuP,
         label=['Signal', 'OOPS', 'OtherCC', 'NuNC', 'Other'], histtype='barstacked' )

ax.set_xlim(-1,1)
ax.set_xlabel('cos('+r'$\theta_{\mu,p}$'+')')
ax.set_ylabel('Count')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
binsCosThMuP = np.linspace(-1.,1.,21)

ax.hist( selectionTree['kRecoCosThMuP'], bins=binsCosThMuP, histtype='step', label='Nominal' )
ax.hist( selectionTree['kRecoCosThMuP'], weights=selectionTree['GENIEReWeight_ICARUS_v2_multisigma_ZExpA1CCQE'][:,2],\
         bins=binsCosThMuP, histtype='step', label='-1'+r'$\sigma$', color='red', linestyle='--' )
ax.hist( selectionTree['kRecoCosThMuP'], weights=selectionTree['GENIEReWeight_ICARUS_v2_multisigma_ZExpA1CCQE'][:,4],\
         bins=binsCosThMuP, histtype='step', label='+1'+r'$\sigma$', color='red', linestyle='--' )

ax.set_xlim(-1,1)
ax.set_xlabel('cos('+r'$\theta_{\mu,p}$'+')')
ax.set_ylabel('Count')
ax.legend()

### Truth tree plot

In [ ]:
true_signal = truthTree[ ak.where(truthTree['SignalType']==1) ]

In [ ]:
fig, ax = plt.subplots()
binsCosThMuP = np.linspace(-1.,1.,21)

ax.hist( true_signal['TrueCosThMuP'], bins=binsCosThMuP, histtype='step' )

ax.set_xlim(-1,1)
ax.set_xlabel('True cos('+r'$\theta_{\mu,p}$'+')')
ax.set_ylabel('Count')

# A look at covariance related to CCQE events and nucleon FSI

In [ ]:
with ur.open('demo.root') as f:
    # CCQE covariance
    hCorr = f['covarTreeCCQE/corr_hist'].to_numpy()
    # CV
    hCV = f['covarTreeCCQE/cvHist'].to_numpy()
    # 100 universes
    hUniv = []
    for i in range(100):
        hUniv.append( f['covarTreeCCQE/hUniverse_'+str(i)].to_numpy() )

binsDeltaPT = np.array( [0., 0.05, 0.10, 0.15, 0.20, 0.25, 0.35, 0.55, 0.85, 2.0] )
centersDeltaPT = np.array( [(binsDeltaPT[i+1]+binsDeltaPT[i])/2. for i in range(len(binsDeltaPT)-1) ] )
widthsDeltaPT = np.array( [(binsDeltaPT[i+1]-binsDeltaPT[i]) for i in range(len(binsDeltaPT)-1) ] )

In [ ]:
fig, ax = plt.subplots()

ax.hist( centersDeltaPT, bins=binsDeltaPT, weights=hCV[0]/(widthsDeltaPT/0.05), histtype='step', label='Nominal' )
for i in range(5):
    ax.hist( centersDeltaPT, bins=binsDeltaPT, weights=hUniv[i][0]/(widthsDeltaPT/0.05),\
             histtype='step', linestyle='--', label=str(i))

ax.set_xlim(0,0.85)
ax.set_ylabel('Count / 0.05 GeV')
ax.set_xlabel(r'$\delta p_T$'+' [GeV]')
ax.legend()

In [ ]:
cov_binning_x = []
cov_binning_y = []
cov_binning_vals = []
for i in range(len(binsDeltaPT)-1):
    xval = centersDeltaPT[i]
    for j in range(len(binsDeltaPT)-1):
        yval = centersDeltaPT[j]
        corrval = hCorr[0][i][j]
        cov_binning_x.append(xval)
        cov_binning_y.append(yval)
        cov_binning_vals.append(corrval)
cov_binning_x = np.array( cov_binning_x )
cov_binning_y = np.array( cov_binning_y )
cov_binning_vals = np.array( cov_binning_vals )

fig, ax = plt.subplots()
im = ax.hist2d( cov_binning_x, cov_binning_y, weights=cov_binning_vals, bins=[binsDeltaPT, binsDeltaPT] )

fig.colorbar(im[3],ax=ax)

ax.set_xlim(0,0.85)
ax.set_ylim(0,0.85)
ax.set_ylabel(r'$\delta p_T$'+' [GeV]')
ax.set_xlabel(r'$\delta p_T$'+' [GeV]')

ax.text(x=0.05,y=0.86,s='FSI N Knob')